In [1]:
import requests
import pandas as pd

In [3]:
df = pd.read_csv('part1_station_data.csv')

In [4]:
df

,latitude,longitude,num_bikes
0,50.961008,6.930710,0
1,50.926356,6.957092,0
2,50.938831,6.906270,1
3,50.972420,6.951578,0
4,50.935485,6.961159,0
...,...,...,...
171,50.919935,7.097071,0
172,50.962241,7.003033,3
173,50.967114,6.989276,0
174,50.930053,6.993490,0


In [5]:
import requests
import pandas as pd

headers = {
    "Accept": "application/json",
    "Authorization": "fsq36xroNyNlJQOc5/NZnRcJEi6Fwt7oBls6wrIyzKhZGAk="
}

foursquare_data = []
for station in df.itertuples():
    params = {
        'll': f'{station.latitude},{station.longitude}',
        'radius': 1000, 
        'categories': '13065,13003',  # Dining and Drinking > Restaurant, Dining and Drinking > Bar
        'limit': 50  
    }
    response = requests.get('https://api.foursquare.com/v3/places/search', headers=headers, params=params)

    if response.status_code != 200:
        print(f"Request failed with status code:{response.status_code}")
        continue

    data = response.json()

    for venue in data['results']:
        venue_info = {
            'name': venue['name'],
            'latitude': venue['geocodes']['main']['latitude'],
            'longitude': venue['geocodes']['main']['longitude'],
            'category': venue['categories'][0]['name'] if venue['categories'] else 'Unknown'
        }
        foursquare_data.append(venue_info)

In [6]:
foursquare_df = pd.DataFrame(foursquare_data)
foursquare_df

,name,latitude,longitude,category
0,Essers Gasthaus,50.956494,6.927119,Austrian Restaurant
1,Pane e Cioccolata,50.954232,6.924428,Pizzeria
2,Vasco da Gama,50.955488,6.932871,Portuguese Restaurant
3,Wicleff,50.956987,6.918339,Wine Bar
4,Bäckerei Pastelaria Luso,50.955228,6.931974,Bakery
...,...,...,...,...
4736,Dolce Gelato,50.930496,6.994907,Bar
4737,Mende Brigitte Gaststätte Taunushof,50.932220,6.996947,Restaurant
4738,Firma G. Streng,50.929615,6.999153,Restaurant
4739,Pizza Taxi Donna Rosa,50.930398,6.999717,Pizzeria


In [7]:
foursquare_df.to_csv('part2_foursquare_data.csv', index=False)

In [9]:
import time
yelp_api_key = 'BISBc6fQXBhVDrEiGFK6WZFSUJWhaneFpm2naqAHSb2nUJjPFZ1xHpdZfvGCMKbRnIRP-KgjFnqDnkk83PkqEHCuKXfZ66DoCi3F6jBzXWgeikfqam0kchu21LRxZXYx'
headers = {'Authorization': f'Bearer {yelp_api_key}'}

yelp_data = []
for station in df.itertuples():
    params = {
        'latitude': station.latitude,
        'longitude': station.longitude,
        'radius': 1000,
        'categories': 'restaurants,bars',
        'limit': 50
    }
    response = requests.get('https://api.yelp.com/v3/businesses/search', headers=headers, params=params)

    if response.status_code == 200:
        data = response.json().get('businesses', [])
        for business in data:
            business_info = {
                'name': business['name'],
                'latitude': business['coordinates']['latitude'],
                'longitude': business['coordinates']['longitude'],
                'rating': business.get('rating', None)
            }
            yelp_data.append(business_info)
    elif response.status_code == 429:
        print("Rate limit reached, request suspended")
        time.sleep(60)  
        continue
    else:
        print(f"Request failed with status code:{response.status_code}, response content:{response.text}")

    time.sleep(1)  

In [10]:
yelp_df = pd.DataFrame(yelp_data)
yelp_df

,name,latitude,longitude,rating
0,Café Franck,50.956350,6.922420,4.3
1,Pane e Cioccolata,50.954260,6.924250,4.3
2,Vasco da Gama,50.956440,6.936020,4.6
3,Shepheard,50.955328,6.932669,4.5
4,Essers Gasthaus,50.956510,6.927020,4.1
...,...,...,...,...
4429,Nimet Chicken,50.937340,6.999210,1.0
4430,Café Atlas,50.930158,6.996726,5.0
4431,Chidoba Mexican Grill,50.938777,6.996378,3.0
4432,Pizza&Döner Mediterrane,50.938387,6.977723,3.6


In [11]:
yelp_df.to_csv('part2_yelp_data.csv', index=False)

In [12]:
len(foursquare_df), len(yelp_df)

(4741, 4434)

In [13]:
top_10_restaurants = yelp_df.sort_values(by='rating', ascending=False).head(10)
print(top_10_restaurants)

                  name   latitude  longitude  rating
2202            Mezi's  50.947211   6.886088     5.0
3825  Mr. Cake Cologne  50.933728   6.960151     5.0
2706        Café Bahar  50.937510   6.892887     5.0
2703      Playa Hütt'n  50.932350   6.871700     5.0
1678      Playa Hütt'n  50.932350   6.871700     5.0
702          Neue Post  50.971300   7.049740     5.0
700        Kreta Grill  50.971370   7.033960     5.0
693   Salva's Pastabar  50.971052   7.073041     5.0
3793  Mr. Cake Cologne  50.933728   6.960151     5.0
682            By Memo  50.974686   7.013613     5.0
